In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SyntaxError: incomplete input (4179869896.py, line 17)

In [ ]:
#import os

#for root, dirs, files in os.walk("/kaggle/input"):
#    for name in files:
#        print(os.path.join(root, name))


In [1]:
import os
base_dir = "/kaggle/working/data"
train_cats = os.path.join(base_dir, "cats")
train_dogs = os.path.join(base_dir, "dogs")

os.makedirs(train_cats, exist_ok=True)
os.makedirs(train_dogs, exist_ok=True)

In [2]:
import shutil
src_dir="/kaggle/input/zip-file/train"
for file in os.listdir(src_dir):
    src_path= os.path.join(src_dir,file)
    if file.startswith("cat"):
        shutil.copy(src_path,train_cats)
    elif file.startswith("dog"):
        shutil.copy(src_path,train_dogs)

In [3]:
print("Cats:", len(os.listdir(train_cats)))
print("Dogs:", len(os.listdir(train_dogs)))


Cats: 12500
Dogs: 12500


In [4]:
import os

base_dir = "/kaggle/working/dataset"
for split in ['train','val']:
    for cls in ['dog','cat']:
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)

IndentationError: unexpected indent (108032753.py, line 2)

In [5]:
import random

In [6]:
def split_data(src_dir, train_dir, val_dir, split_ratio=0.8):
    files = os.listdir(src_dir)
    random.shuffle(files)
    split=int(len(files)*0.8)
    for f in files[:split]:
        shutil.copy(os.path.join(src_dir,f),train_dir)
    for f in files[split:]:
        shutil.copy(os.path.join(src_dir, f), val_dir)

In [7]:
split_data("/kaggle/working/data/cats","/kaggle/working/dataset/train/cat","/kaggle/working/dataset/val/cat")

In [8]:
split_data("/kaggle/working/data/dogs","/kaggle/working/dataset/train/dog","/kaggle/working/dataset/val/dog")

✅ Dataset fully repaired (file + folder safe)


In [9]:
for split in ["train", "val"]:
    print(f"\n{split.upper()}")
    for cls in os.listdir(f"/kaggle/working/dataset/{split}"):
        print(cls, ":", len(os.listdir(f"/kaggle/working/dataset/{split}/{cls}")))



TRAIN
dog : 10000
cat : 10000

VAL
dog : 2500
cat : 2500


✅ Dataset refilled correctly


In [15]:
for split in ["train", "val"]:
    print(f"\n{split.upper()}")
    for cls in os.listdir(f"/kaggle/working/dataset/{split}"):
        print(cls, ":", len(os.listdir(f"/kaggle/working/dataset/{split}/{cls}")))



TRAIN
cat : 10000
dog : 10000

VAL
cat : 2500
dog : 2500


In [10]:
import tensorflow as tf
import keras
img_size=(224,224)
batch_size=32
AUTOTUNE = tf.data.AUTOTUNE

train_ds= keras.utils.image_dataset_from_directory("/kaggle/working/dataset/train",image_size=img_size,batch_size=32,shuffle=True)
val_ds= keras.utils.image_dataset_from_directory("/kaggle/working/dataset/val",image_size=img_size,batch_size=32,shuffle=True)

2025-12-26 06:53:33.396310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766732013.620729      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766732013.702167      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766732014.257009      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766732014.257048      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766732014.257051      55 computation_placer.cc:177] computation placer alr

Found 20000 files belonging to 2 classes.


I0000 00:00:1766732028.849493      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766732028.853402      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 5000 files belonging to 2 classes.


In [11]:
train_ds=train_ds.prefetch(AUTOTUNE)
val_ds=val_ds.prefetch(AUTOTUNE)

In [12]:
from tensorflow.keras.applications import VGG16

In [13]:
base_model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
for layers in base_model.layers:
    layers.trainable=False

In [15]:
from tensorflow.keras.layers import Dense,Flatten,Dropout,BatchNormalization,RandomRotation,RandomZoom,RandomFlip,RandomTranslation,GlobalAveragePooling2D,Activation,Rescaling
from tensorflow.keras import Sequential

In [16]:
model=Sequential()
model.add(Rescaling(1./255, input_shape=(224,224,3)))
model.add(RandomRotation(0.2))
model.add(RandomFlip(0.2))
model.add(RandomZoom(0.1))
model.add(RandomTranslation(width_factor=0.2,height_factor=0.2))
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation              │ (None, 224, 224, 3)    │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,791,617 (56.43 MB)

 Trainable params: 76,481 (298.75 KB)

 Non-trainable params: 14,715,136 (56.13 MB)

In [18]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation              │ (None, 224, 224, 3)    │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,791,617 (56.43 MB)

 Trainable params: 76,481 (298.75 KB)

 Non-trainable params: 14,715,136 (56.13 MB)

In [19]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_accuracy',min_delta=0.001,restore_best_weights=True)]
)


Epoch 1/20


I0000 00:00:1766732151.923733     133 cuda_dnn.cc:529] Loaded cuDNN version 91002


625/625 ━━━━━━━━━━━━━━━━━━━━ 111s 161ms/step - accuracy: 0.6544 - loss: 0.6201 - val_accuracy: 0.8898 - val_loss: 0.2793
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 101s 162ms/step - accuracy: 0.8027 - loss: 0.4269 - val_accuracy: 0.9076 - val_loss: 0.2227
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 102s 163ms/step - accuracy: 0.8258 - loss: 0.3887 - val_accuracy: 0.9120 - val_loss: 0.2097
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 102s 163ms/step - accuracy: 0.8344 - loss: 0.3706 - val_accuracy: 0.9168 - val_loss: 0.1943
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 102s 163ms/step - accuracy: 0.8385 - loss: 0.3600 - val_accuracy: 0.9184 - val_loss: 0.1960
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 102s 163ms/step - accuracy: 0.8378 - loss: 0.3586 - val_accuracy: 0.9166 - val_loss: 0.1927
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 102s 163ms/step - accuracy: 0.8419 - loss: 0.3465 - val_accuracy: 0.9164 - val_loss: 0.1991
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 102s 163ms/step - accuracy: 0.8498 - loss: 0.33

In [20]:
import numpy as np

In [21]:
IMG_SIZE=(224,224)
def predict_image(img):
    img=img.resize(IMG_SIZE)
    img=np.array(img)
    img = np.expand_dims(img, axis=0)
    pred=model.predict(img,verbose=0)[0][0]

    label='DOG🐶' if pred>0.5 else 'CAT🐈‍⬛'
    return label

In [22]:
import gradio as gr
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Textbox(label="Prediction"),
    title="Cat vs Dog Classifier",
    description="CNN trained from scratch with TransferLearning(VGG16)."
)


In [ ]:
interface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://2163913be9318392f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
